In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

# import metrics for evaluation acuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [48]:
# Função para calcular métricas
def calcular_metricas(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }

In [49]:
# extraindo os dados separados de treino e teste

x_train = pd.read_csv('data/x_train.csv')
x_test = pd.read_csv('data/x_test.csv')
Y_train = pd.read_csv('data/y_train.csv')
Y_test = pd.read_csv('data/y_test.csv')

Y_train = np.ravel(Y_train)
Y_test = np.ravel(Y_test)

In [50]:
# realizando a normalização dos dados

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

___
### Modelos avaliados

- K-NN
- Árvore de Decisão
- Random Forest
- XGBoost 
- LightGBM
- MLP (Multilayer Perceptron)
- SVM (Support Vector Machine)
- LVQ (Learning Vector Quantization)
- Comitê de Redes Neurais Artificiais
- Comitê Heterogêneo (Stacking)

Os modelos serão submetidos: 
- busca de hiperparamentos randomizada usando RandomizedSearchCV
- Validação Cruzada de 10 folds
- Métricas de avaliação acurácia, precisão, recall e F1-score (média e desvio padrão)
- Matriz de Confusão
- Curva de Aprendizado
- Comparar as métricas de cada modelo usando Box Plot


In [51]:
# Hiperparâmetros de modelos 

params_knn = {
    'n_neighbors': range(1, 31),  
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

params_decision_tree = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': list(range(3, 21, 2)),
    'min_samples_split': [2, 5, 10, 20],  # Mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4, 8]  # Mínimo de amostras por folha
}

params_random_forest = {
    'n_estimators': range(1, 200, 5),
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(3, 30, 3)),
}

params_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3], # Grau do kernel polinomial
    'gamma': ['scale', 'auto', 0.1, 1]  # Controle de alcance do kernel
}

params_mlp = {
    'hidden_layer_sizes': [(10,), (20,), (50,), (100,), (50, 50), (100, 50), (50, 25), (200, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [200, 500]
}

params_xgboost = {
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.01, 0.1, 0.3, 0.5],  
    'max_depth': range(3, 11),
    'subsample': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 1] }

params_lightgbm = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': range(3, 11),
    'num_leaves': [15, 31, 63],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

In [52]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

models = {
    'K-NN': (KNeighborsClassifier(), params_knn),
    'Decision Tree': (DecisionTreeClassifier(random_state=42), params_decision_tree),
    'Random Forest': (RandomForestClassifier(random_state=42), params_random_forest),
    'SVM': (SVC(random_state=42), params_svm),
    'MLP': (MLPClassifier(random_state=42), params_mlp),
    'XGBoost': (XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), params_xgboost),
    'LightGBM': (LGBMClassifier(random_state=42), params_lightgbm)
}

In [55]:
knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(
            estimator=knn,
            param_grid=params_knn,
            cv=10,  # Validação cruzada com 10 folds
            n_jobs=-1  # Usa todos os processadores disponíveis
        )

# Treinando o modelo com os dados de treino
grid_search_knn.fit(x_train, Y_train)
# Imprimindo os melhores parâmetros e o score
print(f"\nMelhores hiperparâmetros para KNN:")
print(grid_search_knn.best_params_)
print(f"Melhor score (acurácia média com CV=10): {grid_search_knn.best_score_:.4f}")



Melhores hiperparâmetros para KNN:
{'metric': 'manhattan', 'n_neighbors': 30, 'weights': 'distance'}
Melhor score (acurácia média com CV=10): 0.5449


In [56]:
decisiontree_grid = DecisionTreeClassifier(random_state=42)

grid_search_decisiontree = GridSearchCV(
    estimator=decisiontree_grid,
    param_grid=params_decision_tree,
    cv=10,
    n_jobs=-1
)

grid_search_decisiontree.fit(x_train, Y_train)

print(f"\nMelhores hiperparâmetros para Decision Tree:")
print(grid_search_decisiontree.best_params_)
print(f"Melhor score (acurácia média com CV=10): {grid_search_decisiontree.best_score_:.4f}")


Melhores hiperparâmetros para Decision Tree:
{'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 8, 'min_samples_split': 2, 'splitter': 'best'}
Melhor score (acurácia média com CV=10): 0.5714


In [ ]:
decisionTree = DecisionTreeClassifier(random_state=42)
random_search_dt = GridSearchCV(
    estimator=decisionTree,
    param_distributions=params_decision_tree,
    n_iter=10, 
    cv=10,  # Validação cruzada com 10 folds
    random_state=42,
    n_jobs=-1  # Usa todos os processadores disponíveis
)

# Treinando o modelo com os dados de treino
random_search_dt.fit(x_train, Y_train)
# Imprimindo os melhores parâmetros e o score
print(f"\nMelhores hiperparâmetros para KNN:")
print(random_search_dt.best_params_)
print(f"Melhor score (acurácia média com CV=10): {random_search_dt.best_score_:.4f}")


Melhores hiperparâmetros para KNN:
{'splitter': 'best', 'min_samples_split': 10, 'min_samples_leaf': 8, 'max_depth': 7, 'criterion': 'entropy'}
Melhor score (acurácia média com CV=10): 0.5707


In [ ]:
rf = RandomForestClassifier(random_state=42)
random_search_rf = RandomizedSearchCV(
            estimator=rf,
            param_distributions=params_random_forest,
            n_iter=10, 
            cv=10,  # Validação cruzada com 10 folds
            random_state=42,
            n_jobs=-1  # Usa todos os processadores disponíveis
        )

# Treinando o modelo com os dados de treino
random_search_rf.fit(x_train, Y_train)
# Imprimindo os melhores parâmetros e o score
print(f"\nMelhores hiperparâmetros para Random Forest:")
print(random_search_rf.best_params_)
print(f"Melhor score (acurácia média com CV=10): {random_search_rf.best_score_:.4f}")


Melhores hiperparâmetros para Random Forest:
{'n_estimators': 191, 'max_depth': 15, 'criterion': 'gini'}
Melhor score (acurácia média com CV=10): 0.5799


In [1]:
svm = SVC(random_state=42)
random_search_svm = RandomizedSearchCV(
            estimator=svm,
            param_distributions=params_svm,
            n_iter=10, 
            cv=10,  # Validação cruzada com 10 folds
            random_state=42,
            n_jobs=-1  # Usa todos os processadores disponíveis
        )

# Treinando o modelo com os dados de treino
random_search_svm.fit(x_train, Y_train)
# Imprimindo os melhores parâmetros e o score
print(f"\nMelhores hiperparâmetros para SVM:")
print(random_search_svm.best_params_)
print(f"Melhor score (acurácia média com CV=10): {random_search_svm.best_score_:.4f}")

NameError: name 'SVC' is not defined

In [ ]:
mlp = MLPClassifier(random_state=42)
random_search_mlp = RandomizedSearchCV(
            estimator=mlp,
            param_distributions=params_mlp,
            n_iter=10, 
            cv=10,  # Validação cruzada com 10 folds
            random_state=42,
            n_jobs=-1  # Usa todos os processadores disponíveis
        )

# Treinando o modelo com os dados de treino
random_search_mlp.fit(x_train, Y_train)
# Imprimindo os melhores parâmetros e o score
print(f"\nMelhores hiperparâmetros para MLP:")
print(random_search_mlp.best_params_)
print(f"Melhor score (acurácia média com CV=10): {random_search_mlp.best_score_:.4f}")
